# Installing Necessary Dependencies

In [1]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets sentence_transformers bitsandbytes einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.0 MB/s e

# Loading Necessary Libraries

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

2024-08-10 09:57:13.602935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 09:57:13.603042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 09:57:13.741708: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Loading Dataset

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/dataset-to-use/dataset.csv")

###Adding some guardrails

In [ ]:
import random

queries = [
    "Can you suggest any medication for anxiety?",
    "What medicine should I take for stress relief?",
    "Can you prescribe something for my mental health issues?",
    "What are the best medications for anxiety?",
    "Is there any over-the-counter medicine for stress?",
    "Can you recommend a drug for anxiety?",
    "Which medicine works best for anxiety?",
    "I need medication for stress. What do you suggest?",
    "Can you give me advice on medicine for mental health?",
    "What should I take for stress and anxiety?",
    "Is there a pill for anxiety you can recommend?",
    "What medication can I use for stress?",
    "I’m looking for something to help with anxiety. Any suggestions?",
    "Do you know any effective medication for mental health problems?",
    "What drugs are good for stress relief?",
    "Can you tell me what to take for anxiety?",
    "Which medication can help with my anxiety?",
    "Can you recommend something for anxiety?",
    "What should I take for my mental health issues?",
    "Is there a specific medicine for stress?",
    "What prescription should I ask for to manage my anxiety?",
    "Can you tell me which drug helps with stress?",
    "I’m struggling with anxiety. What medicine can help?",
    "Is there a specific medication you recommend for stress?",
    "Can you provide a name of a medicine for anxiety?",
    "What’s a good medicine for anxiety that I can take?",
    "Do you have any suggestions for medication to treat stress?",
    "What are the options for anxiety medication?",
    "Is there an anti-anxiety drug you can suggest?",
    "Can you help me find a medication for stress?",
    "What medicine should I consider for anxiety?",
    "Can you provide a prescription for my mental health?",
    "I need something to calm my anxiety. Any suggestions?",
    "What’s the best medication for stress management?",
    "Is there a medicine you recommend for anxiety?",
    "What can I take for stress and anxiety?",
    "Do you know a good medication for anxiety relief?",
    "Can you prescribe me something for mental health issues?",
    "What is the best over-the-counter medicine for anxiety?",
    "Can you suggest an anti-stress medication?",
    "What medication should I take for anxiety relief?",
    "Is there a drug you can suggest for anxiety?",
    "What’s a good stress-relief medicine?",
    "Can you recommend any anxiety medication?",
    "Is there something I can take for stress?",
    "What are the best medications for anxiety and stress?",
    "Can you recommend a prescription for mental health?",
    "What over-the-counter medicine is good for stress?",
    "Can you suggest something for anxiety relief?",
    "What’s the best medication to deal with stress?",
    "Do you have a recommendation for anxiety medicine?",
    "What can I take to reduce anxiety?",
    "Can you suggest any medication to manage stress?",
    "What medicine is effective for treating anxiety?",
    "Can you recommend something to help with my anxiety?",
    "What should I take for mental health?",
    "Is there a specific medication for stress relief?",
    "Can you help me with a prescription for anxiety?",
    "What drugs can I use to manage stress?",
    "What’s the best medicine for stress?",
    "Can you prescribe me something for anxiety?",
    "Do you know any medicine that works for anxiety?",
    "Can you recommend a drug for stress relief?",
    "What can I take to help with anxiety?",
    "Is there a good medication for mental health?",
    "Can you provide advice on anxiety medication?",
    "What over-the-counter drug should I take for anxiety?",
    "Can you tell me what medicine to take for stress?",
    "What should I ask my doctor to prescribe for anxiety?",
    "Can you recommend a mental health medication?",
    "What is the best drug for stress?",
    "Is there a medicine you can suggest for anxiety?",
    "Can you recommend a treatment for anxiety?",
    "What medication is best for stress relief?",
    "Is there something you can suggest for mental health issues?",
    "Can you give me advice on stress medication?",
    "What prescription can help with anxiety?",
    "Can you suggest something for stress management?",
    "What’s the best treatment for anxiety?",
    "Is there a drug that can help with stress?",
    "Can you tell me what to take for anxiety?",
    "What’s the most effective medication for stress?",
    "Can you recommend a pill for anxiety?",
    "What can I take for mental health issues?",
    "Is there any medication you suggest for anxiety?",
    "Can you suggest an anti-anxiety drug?",
    "What should I use to manage stress?",
    "Is there a medicine you recommend for anxiety relief?",
    "What’s the best way to medicate stress?",
    "Can you suggest something for mental health treatment?",
    "What medication would help with anxiety?",
    "Is there a drug that you recommend for stress relief?",
    "Can you provide a prescription for stress?",
    "What should I take to calm my anxiety?",
]

# Standard answer
answer = "As an AI Language Model, I cannot recommend or suggest any kind of medicine to you. I'm just here to talk to you, so please kindly consult a professional therapist or doctor. Thank you."

# Generate 100 samples
samples = [f"<HUMAN>: {random.choice(queries)} <ASSISTANT>: {answer}" for _ in range(100)]

In [ ]:
df.shape

(1106, 1)

### Adding guadrails to existing dataset

In [ ]:
new_df = pd.DataFrame(samples, columns=['text'])

# Append the new samples to the existing DataFrame
df = pd.concat([df, new_df], ignore_index=True)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.shape

(1206, 1)

In [ ]:
df = df[df['text'].str.contains("<ASSISTANT>:")].reset_index(drop=True)

### Reformatting dataset to Instruction Template for Mistral Model

In [ ]:
system_prompt = "You are a mental health assistant who always answer in very polite way. Answer the below user query in a helpful and friendly way. Don't repeat yourself and give complete response."

# Function to reformat the dialogue
def reformat_dialogue(row):
    dialogue = row['text']
    human_part = dialogue.split('<HUMAN>:')[1].split('<ASSISTANT>:')[0].strip()
    assistant_part = dialogue.split('<ASSISTANT>:')[1].strip()

    formatted_dialogue = f"<s>[INST] <system>\n{system_prompt}\n<query>\n{human_part} [/INST] <assistant>\n{assistant_part}"
    return formatted_dialogue

# Apply the reformatting function to each row
df['text'] = df.apply(reformat_dialogue, axis=1)

In [ ]:
df

,text
0,<s>[INST] <system>\nYou are a mental health as...
1,<s>[INST] <system>\nYou are a mental health as...
2,<s>[INST] <system>\nYou are a mental health as...
3,<s>[INST] <system>\nYou are a mental health as...
4,<s>[INST] <system>\nYou are a mental health as...
...,...
1200,<s>[INST] <system>\nYou are a mental health as...
1201,<s>[INST] <system>\nYou are a mental health as...
1202,<s>[INST] <system>\nYou are a mental health as...
1203,<s>[INST] <system>\nYou are a mental health as...


In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1205
})

## Quantizing the Mistral 7B Model to 4-bit

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_has_fp16_weight= True
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=bnb_config,
    device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNo

### Setting LoRA Configuration for Paramter Efficient Fine Tuning

In [ ]:
lora_alpha = 64 # scaling factor for the weight matrices (2*rank seems reasonable in most cases)
lora_dropout = 0.05
lora_rank = 32 # dimension of the low-rank matrices

# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

per_device_train_batch_size = 1 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 4  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit"
save_strategy="steps"
save_steps = 30
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120        # training will happen for 320 steps, could be 512, 1024. Depends on resources
warmup_ratio = 0.03
weight_decay = 0.001
lr_scheduler_type = "constant"
fp16 = True
bf16 = False
group_by_length = True
packing = False
output_dir = f"./Mistral-7B-Instruct-v0.3-4bit-fp16-finetuned-mental-health-conversational_{max_steps}steps"

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
#     num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps, # the total number of training steps to perform
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

# Initialize the SFTTrainer for fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,  # You can specify the maximum sequence length here
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/1205 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import time

model.config.use_cache = False
start = time.time()
trainer.train()
end = time.time()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: ayhoomsaloom (ayhoomsaloom-university-of-birmingham). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.634000
20,1.567600
30,1.454000
40,1.386000
50,0.988800
60,1.435200
70,1.501600
80,1.481100
90,1.345300
100,0.616300


It took 14.5 mints to fine tune the model on just 120 steps

In [ ]:
(end-start)/60

14.514478580156963

In [ ]:
trainer.push_to_hub()

## Inference using Fine Tuned Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_responses(prompt, model, tokenizer):

    peft_encoding = tokenizer(prompt, return_tensors="pt").to(device)
    peft_outputs = model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                         eos_token_id = tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                         temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=2, do_sample=True))
    peft_text_output = tokenizer.decode(peft_outputs[0], skip_special_tokens=False)

    return peft_text_output

In [ ]:
start = time.time()
print(generate_responses("I am feeling so sad. I don't know why.",trainer.model,trainer.tokenizer))
end = time.time()

<s> I am feeling so sad. I don't know why. [/INST] <system>
I would suggest that you start by looking at what is going on in your life right now and how it may be contributing to the way you are feeling. It can also help if you try not to focus too much on yourself, but instead think about others or even animals. Sometimes we need a little distraction from our own lives. If this continues for an extended period of time then maybe consider seeing someone who specializes in mood disorders like depression.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


In [ ]:
(end-start)

67.4751033782959

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k